# Important Libraries

In [5]:
import pandas as pd
import nltk
import string
import time
from tqdm import tqdm
import numpy as np
import os
import tempfile
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re



# Select Relevent Columns

In [6]:
file_path = r"D:\Study Material\semester 6\Big Data\Assignments\Ass2\dataset.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()

,ARTICLE_ID,TITLE,SECTION_TITLE,SECTION_TEXT
0,0,Anarchism,Introduction,\n\n\n\n\n\n'''Anarchism''' is a political phi...
1,0,Anarchism,Etymology and terminology,\n\nThe term ''anarchism'' is a compound word ...
2,0,Anarchism,History,\n\n===Origins===\nWoodcut from a Diggers docu...
3,0,Anarchism,Anarchist schools of thought,\nPortrait of philosopher Pierre-Joseph Proudh...
4,0,Anarchism,Internal issues and debates,\nconsistent with anarchist values is a contro...


In [7]:
df

,ARTICLE_ID,TITLE,SECTION_TITLE,SECTION_TEXT
0,0,Anarchism,Introduction,\n\n\n\n\n\n'''Anarchism''' is a political phi...
1,0,Anarchism,Etymology and terminology,\n\nThe term ''anarchism'' is a compound word ...
2,0,Anarchism,History,\n\n===Origins===\nWoodcut from a Diggers docu...
3,0,Anarchism,Anarchist schools of thought,\nPortrait of philosopher Pierre-Joseph Proudh...
4,0,Anarchism,Internal issues and debates,\nconsistent with anarchist values is a contro...
...,...,...,...,...
4999995,816930,Vergulde Draeck,The wrecking of the Vergulde Draeck,In 1656 the Vergulde Draeck set sail from the ...
4999996,816930,Vergulde Draeck,Arrival in Batavia,On 7 May 1656 approximately nine days after th...
4999997,816930,Vergulde Draeck,Rescue Attempts,A number of rescue attempts was conducted by t...
4999998,816930,Vergulde Draeck,Discovery of the Vergulde Draeck,The Vergulde Draeck was discovered on 13 April...


In [8]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   ARTICLE_ID     int64 
 1   TITLE          object
 2   SECTION_TITLE  object
 3   SECTION_TEXT   object
dtypes: int64(1), object(3)
memory usage: 152.6+ MB
None


In [9]:
relevant_columns = ['ARTICLE_ID', 'SECTION_TEXT']
df = df[relevant_columns]

In [10]:
df.head()

,ARTICLE_ID,SECTION_TEXT
0,0,\n\n\n\n\n\n'''Anarchism''' is a political phi...
1,0,\n\nThe term ''anarchism'' is a compound word ...
2,0,\n\n===Origins===\nWoodcut from a Diggers docu...
3,0,\nPortrait of philosopher Pierre-Joseph Proudh...
4,0,\nconsistent with anarchist values is a contro...


In [11]:
print(df.isnull().sum())

ARTICLE_ID         0
SECTION_TEXT    3751
dtype: int64


In [12]:
df= df.dropna(subset=['SECTION_TEXT'])

df.head()

,ARTICLE_ID,SECTION_TEXT
0,0,\n\n\n\n\n\n'''Anarchism''' is a political phi...
1,0,\n\nThe term ''anarchism'' is a compound word ...
2,0,\n\n===Origins===\nWoodcut from a Diggers docu...
3,0,\nPortrait of philosopher Pierre-Joseph Proudh...
4,0,\nconsistent with anarchist values is a contro...


In [13]:
print(df.isnull().sum())

ARTICLE_ID      0
SECTION_TEXT    0
dtype: int64


In [14]:
df.to_csv(r"D:\Study Material\semester 6\Big Data\Assignments\Ass2\dataset1.csv", index=False)

# Preprocess Data 

In [2]:
file_path = r"D:\Study Material\semester 6\Big Data\Assignments\Ass2\dataset1.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()

,ARTICLE_ID,SECTION_TEXT
0,0,\n\n\n\n\n\n'''Anarchism''' is a political phi...
1,0,\n\nThe term ''anarchism'' is a compound word ...
2,0,\n\n===Origins===\nWoodcut from a Diggers docu...
3,0,\nPortrait of philosopher Pierre-Joseph Proudh...
4,0,\nconsistent with anarchist values is a contro...


In [7]:
# Set up stop words and punctuation
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)
df.head()

,ARTICLE_ID,SECTION_TEXT
0,0,\n\n\n\n\n\n'''Anarchism''' is a political phi...
1,0,\n\nThe term ''anarchism'' is a compound word ...
2,0,\n\n===Origins===\nWoodcut from a Diggers docu...
3,0,\nPortrait of philosopher Pierre-Joseph Proudh...
4,0,\nconsistent with anarchist values is a contro...


In [8]:
df1=df.head()

In [14]:
# Function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove stop words, punctuation, and special characters
    stop_words = set(stopwords.words('english'))
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    # Remove stop words
    tokens = [token for token in text.split() if token.strip() and token not in stop_words]
    # Join the tokens into a single string
    processed_text = ' '.join(tokens)
    return processed_text

# Function to preprocess text in chunks and write to disk
def preprocess_text_in_chunks(data, chunk_size):
    total_chunks = len(data) // chunk_size + 1
    temp_dir = tempfile.mkdtemp()  # Create a temporary directory to store processed chunks
    chunk_filenames = []  # List to store filenames of processed chunks
    with tqdm(total=total_chunks) as pbar:
        for i, chunk in enumerate(chunk_generator(data, chunk_size)):
            processed_chunk = chunk.copy()
            processed_chunk['SECTION_TEXT'] = processed_chunk['SECTION_TEXT'].apply(preprocess_text)
            chunk_filename = os.path.join(temp_dir, f'chunk_{i}.csv')
            processed_chunk.to_csv(chunk_filename, index=False)
            chunk_filenames.append(chunk_filename)
            pbar.update(1)
    return chunk_filenames

# Function to read and concatenate processed chunks from disk
def concatenate_processed_chunks(chunk_filenames):
    processed_chunks = [pd.read_csv(filename) for filename in chunk_filenames]
    return pd.concat(processed_chunks, ignore_index=True)

# Define chunk generator
def chunk_generator(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i+chunk_size]


        
    
    
# Define chunk size
chunk_size = 1000
total_start_time = time.time()

# Preprocess text in chunks and write to disk
processed_chunk_filenames = preprocess_text_in_chunks(df, chunk_size)

# Read and concatenate processed chunks from disk
processed_df = concatenate_processed_chunks(processed_chunk_filenames)

# Clean up temporary files
for filename in processed_chunk_filenames:
    os.remove(filename)
os.rmdir(os.path.dirname(processed_chunk_filenames[0]))

total_end_time = time.time()


total_time_taken = total_end_time - total_start_time
print("Total time taken for preprocessing: {:.2f} seconds".format(total_time_taken))


100%|██████████████████████████████████████████████████████████████████████████████| 4997/4997 [24:39<00:00,  3.38it/s]


Total time taken for preprocessing: 1593.53 seconds


In [15]:
processed_df.to_csv(r"D:\Study Material\semester 6\Big Data\Assignments\Ass2\finalprepro.csv", index=False)

In [16]:
processed_df

,ARTICLE_ID,SECTION_TEXT
0,0,anarchism political philosophy advocates selfg...
1,0,term anarchism compound word composed word ana...
2,0,origins woodcut diggers document william evera...
3,0,portrait philosopher pierrejoseph proudhon 180...
4,0,consistent anarchist values controversial subj...
...,...,...
4996244,816930,1656 vergulde draeck set sail cape good hope s...
4996245,816930,7 may 1656 approximately nine days loss vergul...
4996246,816930,number rescue attempts conducted dutch east in...
4996247,816930,vergulde draeck discovered 13 april 1963 latit...


## Group by ARTICLE_ID and concatenate the SECTION_TEXT column

In [17]:
file_path = r"D:\Study Material\semester 6\Big Data\Assignments\Ass2\finalprepro.csv"

# Read the CSV file into a DataFrame
df1 = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df1.head()

,ARTICLE_ID,SECTION_TEXT
0,0,anarchism political philosophy advocates selfg...
1,0,term anarchism compound word composed word ana...
2,0,origins woodcut diggers document william evera...
3,0,portrait philosopher pierrejoseph proudhon 180...
4,0,consistent anarchist values controversial subj...


In [18]:
df1['SECTION_TEXT'].dtype

dtype('O')

In [19]:
# Find duplicate ARTICLE_ID values
duplicate_article_ids = df1[df1.duplicated(subset='ARTICLE_ID', keep=False)]['ARTICLE_ID']

# Print the duplicate ARTICLE_ID values
print("Duplicate ARTICLE_ID values:")
print(duplicate_article_ids)


Duplicate ARTICLE_ID values:
0               0
1               0
2               0
3               0
4               0
            ...  
4996244    816930
4996245    816930
4996246    816930
4996247    816930
4996248    816930
Name: ARTICLE_ID, Length: 4959240, dtype: int64


In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4996249 entries, 0 to 4996248
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   ARTICLE_ID    int64 
 1   SECTION_TEXT  object
dtypes: int64(1), object(1)
memory usage: 76.2+ MB


In [21]:
# Check the type of the first entry in the 'SECTION_TEXT' column
entry_type = type(df1.loc[0, 'SECTION_TEXT'])

print("Type of entry:", entry_type)


Type of entry: <class 'str'>


In [22]:
first_df1_row = df1.iloc[0]

In [23]:
print(len(first_df1_row['SECTION_TEXT']))

1022


In [25]:
# Group by ARTICLE_ID and concatenate the SECTION_TEXT column
concatted_df = df1.groupby('ARTICLE_ID')['SECTION_TEXT'].apply(lambda x: ' '.join(str(item) for item in x)).reset_index()


In [26]:
concatted_df.head()

,ARTICLE_ID,SECTION_TEXT
0,0,anarchism political philosophy advocates selfg...
1,1,autism neurodevelopmental disorder characteriz...
2,2,percentage diffusely reflected sunlight relati...
3,3,writing cursive forms named plural aes first l...
4,4,alabama state southeastern region united state...


In [27]:
first_df1_row = concatted_df.iloc[0]

In [28]:
print(len(first_df1_row['SECTION_TEXT']))

52187


In [29]:
concatted_df.to_csv(r'D:\Study Material\semester 6\Big Data\Assignments\Ass2\bestdata.csv', index=False) 


# Creat Sample Data 

In [30]:
file_path = r"D:\Study Material\semester 6\Big Data\Assignments\Ass2\bestdata.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()

,ARTICLE_ID,SECTION_TEXT
0,0,anarchism political philosophy advocates selfg...
1,1,autism neurodevelopmental disorder characteriz...
2,2,percentage diffusely reflected sunlight relati...
3,3,writing cursive forms named plural aes first l...
4,4,alabama state southeastern region united state...


In [31]:
sample_df = df.head(2000)

In [32]:
sample_df

,ARTICLE_ID,SECTION_TEXT
0,0,anarchism political philosophy advocates selfg...
1,1,autism neurodevelopmental disorder characteriz...
2,2,percentage diffusely reflected sunlight relati...
3,3,writing cursive forms named plural aes first l...
4,4,alabama state southeastern region united state...
...,...,...
1995,1995,bernard clairvaux ocist 109020 august 1153 fre...
1996,1996,bishkek formerly pishpek frunze capital larges...
1997,1997,braveheart 1995 american epic war film directe...
1998,1998,brian wilson aldiss obe born 18 august 1925 en...


In [33]:
first_df1_row = sample_df.iloc[0]

In [34]:
print(len(first_df1_row['SECTION_TEXT']))

52187


In [36]:
sample_df.to_csv(r'D:\Study Material\semester 6\Big Data\Assignments\Ass2\sample_data.csv', index=False) 
